In [ ]:
import yfinance as yf
import pandas as pd

# Scarica dati storici per una lista di tickers
def download_sp500_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', auto_adjust=True)
    return {ticker: data[ticker]['Close'].dropna() for ticker in tickers}

# Parametri
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META',  # Tech Giants
    'TSLA', 'NVDA', 'NFLX', 'ADBE', 'CRM',    # Technology & Services
    'JNJ', 'PFE', 'MRK', 'ABBV', 'TMO',       # Healthcare
    'XOM', 'CVX', 'COP', 'PSX', 'SLB',        # Energy
    'BAC', 'JPM', 'C', 'GS', 'MS',            # Financials
    'KO', 'PEP', 'MCD', 'PG', 'WMT'           # Consumer Goods
]

start_date = "2000-01-01"
end_date = "2023-12-31"

# Scarica i dati
sp500_data = download_sp500_data(tickers, start_date, end_date)

[*********************100%***********************]  5 of 5 completed


In [5]:
# Crea segmenti di lunghezza fissa
def create_segments(data, window_size=100, overlap=0.5):
    step = int(window_size * (1 - overlap))  # Calcola passo in base all'overlap
    segments = []
    for ticker, series in data.items():
        series = series.to_numpy()
        for i in range(0, len(series) - window_size + 1, step):
            segment = series[i:i + window_size]
            segments.append({'ticker': ticker, 'segment': segment})
    return segments

# Genera segmenti (50% overlap per aumentare i campioni)
segments = create_segments(sp500_data, window_size=100, overlap=0.5)
print(f"Numero totale di segmenti generati: {len(segments)}")

Numero totale di segmenti generati: 510


In [7]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import os

# Crea directory per salvare le immagini
output_dir = "/Users/roberto/Desktop/spectograms"
os.makedirs(output_dir, exist_ok=True)

# Genera spettrogrammi per i segmenti
def generate_and_save_spectrograms(segments, wavelet='cmor', scale_range=(1, 64)):
    for idx, segment_data in enumerate(segments):
        ticker = segment_data['ticker']
        segment = segment_data['segment']

        # Genera Continuous Wavelet Transform (CWT)
        scales = np.arange(scale_range[0], scale_range[1])
        coefficients, frequencies = pywt.cwt(segment, scales, wavelet)

        # Salva lo spettrogramma
        plt.figure(figsize=(8, 6))
        plt.imshow(
            np.abs(coefficients), 
            extent=[0, len(segment), scale_range[0], scale_range[1]], 
            cmap='jet', 
            aspect='auto', 
            origin='lower'
        )
        plt.colorbar(label='Magnitudo')
        plt.title(f"Spettrogramma - {ticker} - Segment {idx}")
        plt.xlabel("Tempo")
        plt.ylabel("Scala")
        
        # Salva l'immagine
        filename = f"{output_dir}/{ticker}_segment_{idx}.png"
        plt.savefig(filename)
        plt.close()

# Genera spettrogrammi
generate_and_save_spectrograms(segments)

/Users/roberto/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow the format cmorB-C, where B and C are floats representing the bandwidth frequency and center frequency, respectively (example, for backward compatibility: cmor = cmor1.0-0.5).
  wavelet = DiscreteContinuousWavelet(wavelet)
